<a href="https://colab.research.google.com/github/vijaysrajan/buysellconnect/blob/main/basicgemma2blocally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages with compatible versions
!pip install --upgrade transformers>=4.38.0
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade accelerate
!pip install langchain>=0.1.0
!pip install langchain-huggingface
!pip install huggingface_hub>=0.20.0


In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [ ]:

# Step 2: Restart runtime after installation (important!)
# Go to Runtime > Restart runtime in Colab, then run the cells below


In [ ]:

# Step 3: Authenticate with Hugging Face (Gemma is a gated model)
from huggingface_hub import notebook_login
notebook_login()  # This will prompt you to enter your HF token


In [ ]:

# Step 4: Import required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [ ]:

# Step 5: Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [ ]:

# Step 6: Load the model and tokenizer locally
model_id = "google/gemma-2b-it"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)


Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:

# # Detect if CUDA is available and set device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Step 7: Create a pipeline
# text_generation_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=1000,
#     do_sample=False,  # Greedy decoding
#     return_full_text=False,
#     device=1 if device == "cuda" else -1
# )




# Detect if CUDA is available and set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device="auto"
# Step 7: Create a pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    do_sample=False,  # Greedy decoding
    return_full_text=False,
    device=0 if device == "cuda" else -1  # Use GPU 0 if available, else CPU
)

Device set to use cpu


In [ ]:
# Step 8: Create LangChain wrapper
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Step 9: Test the model
print("Testing the model...")
#prompt = "Explain what is Generative AI in 3 bullet points"
prompt = """ \
### Input: \
I want to sell my 2 year old fridge. It is 350 litres and red in colour. I am moving out of my town. \
### Schema:\
{
  "item": "string",
  "quantity": "string",
  "colour": "string"
  "capacity": "string"\
} \
### Response: \
"""


response = llm.invoke(prompt)
print("Response:")
print(response)

# Alternative: Direct pipeline usage (without LangChain)
print("\n" + "="*50)
print("Alternative: Direct pipeline usage")
result = text_generation_pipeline(prompt)
print(result[0]['generated_text'])

Testing the model...
Response:

**Item:** Fridge
**Quantity:** 1
**Colour:** Red
**Capacity:** 350

Alternative: Direct pipeline usage

**Item:** Fridge
**Quantity:** 1
**Colour:** Red
**Capacity:** 350


In [ ]:
Q1 = "I want to sell my 2 year old fridge. It is 350 litres and red in colour. I am moving out of my town."


Q = Q1

In [ ]:
prompt_template1 = ChatPromptTemplate.from_template(f"""
<bos><start_of_turn>user
Answer the following question based on the provided context. If you cannot answer the question based on the context, say "I don't have enough information to answer that question."

Context: Is the question from a buyer or seller or a lessor or renter or is the question from someone who wants to connect. Only 5 choices possible buyer, seller, connector, lessor, renter.

Question: {Q}
<end_of_turn>
<start_of_turn>model
""")
print (prompt_template1.format_messages(Q=Q))

[HumanMessage(content='\n<bos><start_of_turn>user\nAnswer the following question based on the provided context. If you cannot answer the question based on the context, say "I don\'t have enough information to answer that question."\n\nContext: Is the question from a buyer or seller or a lessor or renter or is the question from someone who wants to connect. Only 5 choices possible buyer, seller, connector, lessor, renter.\n\nQuestion: I want to sell my 2 year old fridge. It is 350 litres and red in colour. I am moving out of my town.\n<end_of_turn>\n<start_of_turn>model\n', additional_kwargs={}, response_metadata={})]


In [ ]:
prompt_template2 = ChatPromptTemplate.from_template(f"""
<bos><start_of_turn>user
Answer the following question based on the provided context. If you cannot answer the question based on the context, say "I don't have enough information to answer that question."

Context: Please extract the attributes and features in the question and put it in a json format. Please

Question: {Q}
<end_of_turn>
<start_of_turn>model
""")

print (prompt_template2.format_messages(Q=Q))

[HumanMessage(content='\n<bos><start_of_turn>user\nAnswer the following question based on the provided context. If you cannot answer the question based on the context, say "I don\'t have enough information to answer that question."\n\nContext: Please extract the attributes and features in the question and put it in a json format. Please\n\nQuestion: I want to sell my 2 year old fridge. It is 350 litres and red in colour. I am moving out of my town.\n<end_of_turn>\n<start_of_turn>model\n', additional_kwargs={}, response_metadata={})]


In [ ]:
qa_rag_chain = (
    # Removed commented out code and fixed syntax
    {"question": RunnablePassthrough()}
    | prompt_template1
    | llm
    | StrOutputParser()
)

# Assuming 'question' variable is defined elsewhere or should be Q
# Using Q based on previous cell execution
response = qa_rag_chain.invoke({"question": Q})
print(response)

The question is from a seller. It is asking about the sale of a 2 year old fridge.


**Note:** You need to restart the runtime after installing the library for the changes to take effect. Go to `Runtime > Restart runtime` in the Colab menu.